<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Машинное обучение и майнинг данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# Занятие 8. Apache Spark
## Часть 3. Подсчет числа слов в документе

#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
#### Подсчитаем самые частые слова в  [полном собрании сочинений Уильяма Шекспира ](http://www.gutenberg.org/ebooks/100)([проект Гутенберга](http://www.gutenberg.org/wiki/Main_Page)).  
#### **План: **
#### *Часть 1:* Создание RDD и парных RDD
#### *Часть 2:* Подсчет слов с помощью парных RDD
#### *Часть 3:* Нахождение уникальных слов
#### *Часть 4:* Подсчет слов в файле

### ** Часть 1: Создание RDD и парных RDD**

#### ** (1a) Создание RDD **
#### Метод `sc.parallelize`.

In [1]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
# Print out the type of wordsRDD
print(type(wordsRDD))

<class 'pyspark.rdd.RDD'>


#### ** (1b) Функция map **
#### Напишем функцию и используем map(), чтобы применить ее ко всему набору данных

In [2]:
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  No attempt is made to follow proper
        pluralization rules.

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return word + 's'

print(makePlural('cat'))

cats


#### ** (1c) Применение `makePlural` к RDD **
#### Используем [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) и [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect).

In [5]:
pluralRDD = wordsRDD.map(makePlural)
print(pluralRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


#### ** (1d)  `map` с  `lambda`-функцией **

In [8]:
pluralLambdaRDD = wordsRDD.map(lambda word: word + 's')
print(pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


#### ** (1e) Длины слов **
#### Используем `map()` и `lambda`-функцию для подсчета длины каждого слова

In [12]:
pluralLengths = (pluralRDD
                 .map(lambda word: len(word))
                 .collect())

#### ** (1f) Парные RDD **
#### Парный RDD - это такой RDD, каждый элемент которого - кортеж `(k, v)`  - "ключ-значение". Здесь мы создаем пары (<слово>, 1), чтобы потом просуммировать все единицы. 

In [10]:
wordPairs = wordsRDD.map(lambda word: (word, 1))
print wordPairs.collect()

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


### ** Часть 2: Подсчет слов с помощью парных RDD**

#### ** (2a) Подход на основе `groupByKey()`  **
#### Здесь используем преобразование [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey). Порождаются пары вида `('<слово>', <итератор>)`.

In [12]:
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))

rat: [1, 1]
elephant: [1]
cat: [1, 1]


#### ** (2b)  `groupByKey()` для подсчета числа слов **
#### Теперь суммируем с помощью преобразования `map()`.

In [14]:
wordCountsGrouped = wordsGrouped.map(lambda (word, ones): (word, sum(ones)))
print wordCountsGrouped.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


#### ** (2c) Подсчет с помощью `reduceByKey` **
#### Лучше использовать метод  [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey), который объединяет кортежи с одинаковым ключом и применяет к ним некоторую функцию. И так итеративно, пока каждому ключу не будет соответствовать ровно одно значение. Метод легко масштабируется, поскольку указанную операцию можно выполнять распределенно на разных машинах.

In [16]:
# Note that reduceByKey takes in a function that accepts two values and returns a single value
wordCounts = wordPairs.reduceByKey(lambda a, b: a + b)
print(wordCounts.collect())

[('rat', 2), ('elephant', 1), ('cat', 2)]


#### ** (2d) Все вместе **

In [18]:
wordCountsCollected = (wordsRDD
                       .map(lambda word: (word, 1))
                       .reduceByKey(lambda a,b: a + b)
                       .collect())
print(wordCountsCollected)

[('rat', 2), ('elephant', 1), ('cat', 2)]


### ** Part 3: Нахождение уникальных слов **

#### ** (3a) Уникальные слова **

In [20]:
uniqueWords = wordCounts.count()
print(uniqueWords)

3


#### ** (3b) Метод `reduce` **
#### Среднее число появлений каждого уникального слова

In [22]:
from operator import add
totalCount = (wordCounts
              .map(lambda (word, count): count)
              .reduce(add))
average = totalCount / float(wordCounts.count())
print(totalCount)
print(round(average, 2))

5
1.67


### ** Часть 4: Подсчет числа слов в файле **

#### Завершим создание функции для подсчета слов в файле. Учтем регистр слов и пунктуацию.

#### ** (4a) Функция `wordCount`  **

In [24]:
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return wordListRDD.map(lambda word: (word, 1)).reduceByKey(add)

print(wordCount(wordsRDD).collect())

[('rat', 2), ('elephant', 1), ('cat', 2)]


#### ** (4b) Регистр и пунктуация **
 
#### Функция `removePunctuation` приводит все слова к нижнему регистру, удаляет пунктуацию и пробелы в начале и конце сло.  Используем модуль [re](https://docs.python.org/2/library/re.html) для удаления любых символов, отличных от букв, чисел и знака пробела.

In [26]:
import re

def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub('[^a-zA-Z 0-9]','',text.lower().strip())

print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))

hi you
no underscore


#### ** (4c) Загрузка файла **
#### Для превращения текстового файла в RDD используем метод `SparkContext.textFile()` . 

In [28]:
shakespeareRDD = (sc
                  .textFile('../data/shakespeare.txt')
                  .map(removePunctuation))
print('\n'.join(shakespeareRDD
                .zipWithIndex()  # to (line, lineNum)
                .map(lambda (l, num): '{0}: {1}'.format(num, l))  # to 'lineNum: line'
                .take(15)))

0: 1609
1: 
2: the sonnets
3: 
4: by william shakespeare
5: 
6: 
7: 
8: 1
9: from fairest creatures we desire increase
10: that thereby beautys rose might never die
11: but as the riper should by time decease
12: his tender heir might bear his memory
13: but thou contracted to thine own bright eyes
14: feedst thy lights flame with selfsubstantial fuel


#### ** (4d) Извлечение слов из строк **
#### Подзадачи:
  + #### Каждую строку разбиваем по пробелам;
  + #### Удаляем пустые строки.

In [29]:
shakespeareWordsRDD = shakespeareRDD.flatMap(lambda s: s.split(' '))
shakespeareWordCount = shakespeareWordsRDD.count()
print(shakespeareWordsRDD.top(5))
print(shakespeareWordCount)

[u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds']
928908


#### ** (4e) Удаление "пустых" слов  **
#### Используем метод `filter`

In [31]:
shakeWordsRDD = shakespeareWordsRDD.filter(lambda elem: elem != '')
shakeWordCount = shakeWordsRDD.count()
print(shakeWordCount)

882996


#### ** (4f) Подсчет слов **
#### Применяем функцию `wordCount()`, затем  `takeOrdered()` для выведения самых частых слов. Для `takeOrdered()` нужна своя функция сортировки, поскольку каждый элемент `shakeWordsRDD` - это кортеж. 
#### Появляются стоп-слова. Чаще всего при анализе текстов их игнорируют. 

In [33]:
top15WordsAndCounts = (shakeWordsRDD
                       .map(lambda word: (word, 1))
                       .reduceByKey(add)
                       .takeOrdered(15, key=lambda (w, c): -c))
print(top15WordsAndCounts)
print('\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), top15WordsAndCounts)))

[(u'the', 27361), (u'and', 26028), (u'i', 20681), (u'to', 19150), (u'of', 17463), (u'a', 14593), (u'you', 13615), (u'my', 12481), (u'in', 10956), (u'that', 10890), (u'is', 9134), (u'not', 8497), (u'with', 7771), (u'me', 7769), (u'it', 7678)]
the: 27361
and: 26028
i: 20681
to: 19150
of: 17463
a: 14593
you: 13615
my: 12481
in: 10956
that: 10890
is: 9134
not: 8497
with: 7771
me: 7769
it: 7678
